In [1]:
using Random
#using Printf
using LinearAlgebra
using Statistics
import Tensor_FixedSeedMC as TTMC

In [2]:
function asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)
    tmp1 = mu * T / N_STEPS
    tmp2 = exp(-r * T)
    tmp3 = sqrt(T / N_STEPS)
    d_s = zeros(Float64, N_PATHS)
    
    for i in 1:N_PATHS
        s_curr = S0
        running_average = 0.0
        
        for n in 1:N_STEPS
            # Update the current price
            s_curr += tmp1 * s_curr + sigma * s_curr * tmp3 * d_normals[i + (n - 1) * N_PATHS]
            
            # Update the running average
            running_average += (s_curr - running_average) / n
            
            # Check the barrier
            if running_average <= B
                break
            end
        end
        
        # Calculate payoff
        payoff = max(running_average - K, 0)
        
        # Discounted payoff
        d_s[i] = tmp2 * payoff
    end

    return mean(d_s), 1.96 * std(d_s)/sqrt(N_PATHS)
end


asian_barrier_option (generic function with 1 method)

In [3]:
# Example usage
N_STEPS = 365
N_PATHS = 100000
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05
Random.seed!(1234)
S0_range = collect(90:0.3:119.7) # 200
sigma_range = collect(0.15:0.001:0.249) #200
# オプション価格を格納する行列
option_price = zeros(length(sigma_range), length(S0_range))
d_normals = randn(Float64, N_STEPS * N_PATHS)  # Standard normal random numbers

mean_ds, err = asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

(18.7844348458054, 0.13263036072191842)

In [4]:
@time asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

  0.185858 seconds (4 allocations: 781.344 KiB)


(18.7844348458054, 0.13263036072191842)

In [5]:
asian_barrier_option(T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS)

(18.7844348458054, 0.13263036072191842)

In [6]:
random_combinations = TTMC.generate_random_combinations(length(S0_range), 2, 100);

In [7]:
price = []
errs = []
for i in random_combinations
    S0 = S0_range[i[1]]
    sigma = sigma_range[i[2]]
    option_price, err = asian_barrier_option(K, S0, r, sigma, T, M=M, I=I)
    push!(price, option_price)
    push!(errs, err)
end

res = [price, errs]


UndefVarError: UndefVarError: `M` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
using JLD2
JLD2.@save "price_matrix_asianbarrier_N_PATHS$N_PATHS.jld2" res

UndefVarError: UndefVarError: `res` not defined in `Main`
Suggestion: check for spelling errors or missing imports.